In [456]:
import pymupdf
import mysql.connector
import pandas as pd
import re

In [475]:
mydb = mysql.connector.connect(
    host='192.168.0.41',
    user='lars',
    password='Lilleaker01',
    database='tables',
)

cs = mydb.cursor()

In [172]:
def str_to_number(s):
    try:
        # Replace comma with dot for float conversion
        s = s.replace(',', '.')
        if '.' in s:
            return float(s)
        else:
            return int(s)
    except (ValueError, AttributeError):
        return s


In [145]:
doc = pymupdf.open('ns-en-1995-1-1_2004+a2_2014+na_2024_en_001.pdf')

In [482]:
page = doc[122]
tables = page.find_tables()

table = tables[0]
table

In [491]:
# Extract table caption above the table
bbox = list(table.bbox)

bbox[0] = 72
bbox[2] = 524

bbox[3] = bbox[1]
bbox[1] -= 23

rec = pymupdf.Rect(bbox)

caption = page.get_text('text', clip=rec)[:-2]

caption

'Table 9.1 – Maximum effective flange widths due to the effects of shear lag and plate \nbuckling'

In [492]:
db = table.extract()

db

[['Flange material', 'Shear lag', 'Plate buckling'],
 ['Plywood, with grain direction in\nthe outer plies:\n− Parallel to the webs\n− Perpendicular to the webs',
  '0,1A\n0,1A',
  '20h\nBfB\n25h\nBfB'],
 ['Oriented strand board', '0,15A', '25h\nBfB'],
 ['Particleboard or fibreboard\nwith random fibre orientation',
  '0,2A',
  '30h\nBfB']]

In [ ]:
# Create SQL table name and fields

# Name the table based on the caption
name = caption.split(' – ')[1].replace(' ', '_').lower().replace('-', '_').replace('\n', '_').replace('-\n', '')

# Create fields based on the first row and/or second row of the table
fields = []
for i, val in enumerate(db[0]):
    val = re.sub(r'\(.*\)', '', val)  # Remove any text in parentheses
    if val is not None:
        fields.append(val.replace(' ', '_').replace('-\n', '').replace('\n', '_').replace('-', '_').replace('/', '_or_'))
    else:
        fields.append(fields[-1])

idx = 1
if None in db[1]:
    idx = 2
    for i in range(len(db[1])):
        if db[1][i] is not None:
            fields[i] = fields[i] + '_' + db[1][i].replace(' ', '_').replace('-\n', '').replace('\n', '_').replace('–', '_').replace('/', '_or_')

# Determine the data types based on the third row of the table
types = []
fields_with_types = []
for i, val in enumerate(db[idx]):
    val = str_to_number(val.split('\n')[0])
    if isinstance(val, int):
        types.append(int)
        fields_with_types.append(fields[i] + ' INT')
    elif isinstance(val, float): # or val == '–':
        types.append(float)
        fields_with_types.append(fields[i] + ' DOUBLE')
    else:
        types.append(str)
        fields_with_types.append(fields[i] + ' VARCHAR(255)')

display(name, fields_with_types)

'maximum_effective_flange_widths_due_to_the_effects_of_shear_lag_and_plate_\nbuckling'

['Flange_material VARCHAR(255)',
 'Shear_lag VARCHAR(255)',
 'Plate_buckling VARCHAR(255)']

In [495]:
cs.execute(f'DROP TABLE IF EXISTS {name}')

cs.execute(f'CREATE TABLE IF NOT EXISTS {name} ({", ".join(fields_with_types)})')

ProgrammingError: 1059 (42000): Identifier name 'maximum_effective_flange_widths_due_to_the_effects_of_shear_lag_and_plate_' is too long

In [479]:
db = table.extract()

# Fill the table with data
for row in db[idx:7]:
    nrows = 1
    for i, val in enumerate(row):
        if not isinstance(str_to_number(val), types[i]):
            nrows = len(val.split('\n'))

    for i, val in enumerate(row):
        if '\n' in val:
            val = val.split('\n')
            if len(val) < nrows:
                row[i] = ' '.join(val)
            elif len(val) > nrows:
                for j in range(1, len(val)):
                    val[j] = val[0] + ' ' + val[j]
                row[i] = val[1:]
            else:
                row[i] = val

    data = []
    for i in range(nrows):
        new_data = []
        for j, val in enumerate(row):
            if isinstance(val, list):
                val = str_to_number(val[i])
            else:
                val = str_to_number(val)

            if not isinstance(val, types[j]):
                if types[j] == int or types[j] == float:
                    val = None

            new_data.append(val)

        data.append(new_data)

    cs.executemany(f'INSERT INTO {name} ({", ".join(fields)}) VALUES ({", ".join(["%s"] * len(fields))})', data)

mydb.commit()

In [152]:
data = table.extract()

for i, x in enumerate(data[2][2:], 2):
    data[2][i] = float(x.replace(',', '.'))

data[2]

['Solid timber', 'EN 14081-1', 0.6, 0.8, 2.0]

In [154]:
cs.execute('DROP TABLE test')

In [155]:
cs.execute(f"CREATE TABLE IF NOT EXISTS test(Material VARCHAR(255), Standard VARCHAR(255), Service_class_1 DOUBLE, Service_class_2 DOUBLE, Service_class_3 DOUBLE);")

In [156]:
cs.execute('INSERT INTO test (Material, Standard, Service_class_1, Service_class_2, Service_class_3) VALUES (%s, %s, %s, %s, %s)', data[2])

In [157]:
cs.execute("SHOW TABLES")

for x in cs:
    print(x)

('test',)
